This Project is about building a database in Postgre SQl Database for reporting different type of crime in Boston City. Through this project I have explored how to design schema, data bases and tables , creating role and granting access to the roles.Boston CSV File has been used as a Dataset for building tables in Postgre SQl Database. Here is the Data Dictionary for the columns:
* incident_number - A unique number for incidents
* offense_code - Code for Offense type 
* description - Description for the crime 
* date - Date at which Crime has occured
* day_of_the_week - Day of the week when crime occured
* lat - lat details
* long - long details 

# Lets Start with Library installation for Postgre SQL

In [1]:
! pip install psycopg2-binary

# Connection creation for Postgre SQL DB and Database creation

In [2]:
import psycopg2
conn = psycopg2.connect(dbname ="AJAY_POSTGRE_DATABASE",user = "postgres",password = "Neha@1234")
conn.autocommit = True
cur = conn.cursor()
cur.execute("CREATE DATABASE crime_db8;")
conn.commit()
conn.close()

# Schema creation

In [3]:
import psycopg2
conn = psycopg2.connect(dbname ="crime_db8",user = "postgres",password = "Neha@1234")
conn.autocommit = True
cur = conn.cursor()
cur.execute("CREATE SCHEMA crimes;")
conn.commit()
conn.close()

# Import csv file

In [4]:
import csv
with open(r"C:\Users\ajaya\Desktop\boston.csv") as file:
    reader = csv.reader(file)
    col_headers = next(reader)
    first_row = next(reader)

# Creating auxillary function to get column header and print length of values

In [5]:
def get_col_set(csv_file, col_index):
    import csv 
    values = set()
    with open(csv_file,'r') as f:
        next(f)
        reader = csv.reader(f)
        for row in reader:
            values.add(row[col_index])
        
        return values
    
for i in range(len(col_headers)):
    values = get_col_set(r"C:\Users\ajaya\Desktop\boston.csv", i)
    print(col_headers[i], len(values), sep='\t')
    

incident_number	298329
offense_code	219
description	239
date	1177
day_of_the_week	7
lat	18177
long	18177


In [6]:
print(col_headers)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']


In [7]:
descriptions = get_col_set(r"C:\Users\ajaya\Desktop\boston.csv", 2) # description is at index number 2
max_len = 0
for description in descriptions:
    max_len = max(max_len, len(description))
print(max_len)
    

58


In [8]:
print(col_headers)
print(first_row)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']
['1', '619', 'LARCENY ALL OTHERS', '2018-09-02', 'Sunday', '42.35779134', '-71.13937053']


# Table creation and load data into postgre tables

In [9]:
conn = psycopg2.connect(dbname ="crime_db8",user = "postgres",password = "Neha@1234")
conn.autocommit = True
cur = conn.cursor()
cur.execute("""
    CREATE TYPE weekday AS ENUM ('Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday');
""")
# Create the table.
cur.execute("""
    CREATE TABLE crimes.boston_crimes1 (
        incident_number INTEGER,
        offense_code INTEGER,
        description VARCHAR(100),
        date DATE,
        day_of_the_week weekday,
        lat decimal,
        long decimal
    );
""")
conn.commit()
conn.close()

In [10]:
conn = psycopg2.connect(dbname ="crime_db8",user = "postgres",password = "Neha@1234")
conn.autocommit = True
cur = conn.cursor()
with open(r"C:\Users\ajaya\Desktop\boston.csv") as f:
     cur.copy_expert("COPY crimes.boston_crimes1 FROM STDIN WITH CSV HEADER;", f)
cur.execute("SELECT * FROM crimes.boston_crimes1")
cur.fetchone()
print(len(cur.fetchall()))
conn.commit()
conn.close()

298328


# Boston CSV file was loaded to postgre SQL . 298328 records were loaded in crimes table

# Revoke public access on all schema and databases 

In [11]:

conn = psycopg2.connect(dbname ="crime_db8",user = "postgres",password = "Neha@1234")
conn.autocommit = True
cur = conn.cursor()
cur.execute("REVOKE ALL ON SCHEMA public FROM public;")
cur.execute("REVOKE ALL ON DATABASE crime_db8 FROM public;")
conn.commit()
conn.close()

# Grant access on schema for readonly and readwrite group. Grant select ,insert,delete ,update access to all table to readwrite group.

In [12]:
conn = psycopg2.connect(dbname ="crime_db8",user = "postgres",password = "Neha@1234")
conn.autocommit = True
cur = conn.cursor()
#cur.execute("CREATE GROUP readonly NOLOGIN;")
#cur.execute("CREATE GROUP readwrite NOLOGIN;")
cur.execute("GRANT CONNECT ON DATABASE crime_db8 TO readonly;")
cur.execute("GRANT CONNECT ON DATABASE crime_db8 TO readwrite;")
cur.execute("GRANT USAGE ON SCHEMA crimes TO readonly;")
cur.execute("GRANT USAGE ON SCHEMA crimes TO readwrite;")
cur.execute("GRANT SELECT ON ALL TABLES IN SCHEMA crimes TO readonly;")
cur.execute("GRANT SELECT,INSERT,DELETE,UPDATE ON ALL TABLES IN SCHEMA crimes TO readwrite;")
conn.commit()
conn.close()


# Create Data analyst and Data Scientist user and grant them readonly and readwrite access.

In [13]:
conn = psycopg2.connect(dbname ="crime_db8",user = "postgres",password = "Neha@1234")
conn.autocommit = True
cur = conn.cursor()
cur.execute("CREATE USER data_analyst WITH PASSWORD 'datanalyst';")
cur.execute("CREATE USER data_scientist WITH PASSWORD 'datascientist';")
cur.execute("GRANT readonly TO data_analyst;")
cur.execute("GRANT readwrite TO data_scientist;")
conn.commit()
conn.close()

DuplicateObject: role "data_analyst" already exists


# Final testing for all the roles

In [ ]:
conn = psycopg2.connect(dbname ="crime_db8",user = "postgres",password = "Neha@1234")
conn.autocommit = True
cur = conn.cursor()
# Check users and groups.
cur.execute("""
    SELECT rolname, rolsuper, rolcreaterole, rolcreatedb, rolcanlogin FROM pg_roles
    WHERE rolname IN ('readonly', 'readwrite', 'data_analyst', 'data_scientist');
""")
for user in cur:
    print(user)
print()
# check privileges
cur.execute("""
    SELECT grantee, privilege_type
    FROM information_schema.table_privileges
    WHERE grantee IN ('readonly', 'readwrite');
""")
for user in cur:
    print(user)
conn.close()